<a href="https://colab.research.google.com/github/HelloSriram22/PYTHON-CODES/blob/main/RamaStock2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install spark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=139ef1137b9664ea283511bdb7e461c21106c5c3da1ac834f7f15f65d2b912a0
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [ ]:
!apt-get update # Update apt-get repository.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null # Install Java.
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz # Download Apache Sparks.
!tar xf spark-3.1.1-bin-hadoop3.2.tgz # Unzip the tgz file.
!pip install -q findspark # Install findspark. Adds PySpark to the System path during runtime.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [50.4 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,074 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main 

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"


In [ ]:

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving NYSE_daily_File to NYSE_daily_File


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Define the function to convert string to float
def convert_to_float(num):
    return float(num) if num is not None else None

# Define the SparkSession
spark = SparkSession.builder.appName("Rama_StockInfo").getOrCreate()

# Read text data
read_text_data = spark.read.text("/content/NYSE_daily_File")
rowrdd = read_text_data.rdd.map(lambda x: x['value'])
split_rdd = rowrdd.map(lambda x: x.split('\t'))

# Define the schema
schema = StructType([
    StructField("stock_exchange", StringType(), True),
    StructField("company_symbol", StringType(), True),
    StructField("date", StringType(), True),
    StructField("open_price_of_the_day", FloatType(), True),
    StructField("high_of_the_day", FloatType(), True),
    StructField("low_of_the_day", FloatType(), True),
    StructField("close_of_the_day", FloatType(), True),
    StructField("volume", FloatType(), True),
    StructField("adjusted_close_price", FloatType(), True)
])

# Map to convert string values to appropriate types
new_rdd = split_rdd.map(lambda x: (
    x[0],
    x[1],
    x[2],
    convert_to_float(x[3]),
    convert_to_float(x[4]),
    convert_to_float(x[5]),
    convert_to_float(x[6]),
    convert_to_float(x[7]),
    convert_to_float(x[8])
))

# Create DataFrame
stock_df = spark.createDataFrame(new_rdd, schema)

# Drop rows with null values
cleaned_stock_details = stock_df.dropna()

# Filter for valid values
df2 = cleaned_stock_details.filter("open_price_of_the_day > 0 and high_of_the_day > 0")
df2.show()

# Register DataFrame as a temporary view
df2.createOrReplaceTempView("stock")

# Run SQL query to filter data
filtered_data = spark.sql("SELECT stock_exchange, company_symbol FROM stock WHERE open_price_of_the_day > 68 AND high_of_the_day < 70")
filtered_data.show()
output_path = "output1"
#Stock_df.write.format('orc').save("output1")

# Stop SparkSession
spark.stop()


+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|stock_exchange|company_symbol|      date|open_price_of_the_day|high_of_the_day|low_of_the_day|close_of_the_day|   volume|adjusted_close_price|
+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|          NYSE|           CLI|2009-12-31|                35.39|           35.7|          34.5|           34.57| 890100.0|               34.12|
|          NYSE|           CLI|2009-12-30|                35.22|          35.46|         34.96|            35.4| 516900.0|               34.94|
|          NYSE|           CLI|2009-12-29|                35.69|          35.95|         35.21|           35.34| 556500.0|               34.88|
|          NYSE|           CLI|2009-12-28|                35.67|          36.23|         35.49|           35.69| 565000.0|              

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Define the function to convert string to float
def convert_to_float(num):
    return float(num) if num is not None else None

# Define the SparkSession
spark = SparkSession.builder.appName("Rama_StockInfo").getOrCreate()

# Read text data
read_text_data = spark.read.text("/content/NYSE_daily_File")
rowrdd = read_text_data.rdd.map(lambda x: x['value'])
split_rdd = rowrdd.map(lambda x: x.split('\t'))

# Define the schema
schema = StructType([
    StructField("stock_exchange", StringType(), True),
    StructField("company_symbol", StringType(), True),
    StructField("date", StringType(), True),
    StructField("open_price_of_the_day", FloatType(), True),
    StructField("high_of_the_day", FloatType(), True),
    StructField("low_of_the_day", FloatType(), True),
    StructField("close_of_the_day", FloatType(), True),
    StructField("volume", FloatType(), True),
    StructField("adjusted_close_price", FloatType(), True)
])

# Map to convert string values to appropriate types
new_rdd = split_rdd.map(lambda x: (
    x[0],
    x[1],
    x[2],
    convert_to_float(x[3]),
    convert_to_float(x[4]),
    convert_to_float(x[5]),
    convert_to_float(x[6]),
    convert_to_float(x[7]),
    convert_to_float(x[8])
))

# Create DataFrame
stock_df = spark.createDataFrame(new_rdd, schema)

# Drop rows with null values
cleaned_stock_details = stock_df.dropna()

# Filter for valid values
df2 = cleaned_stock_details.filter("open_price_of_the_day > 0 and high_of_the_day > 0")
df2.show()

# Register DataFrame as a temporary view
df2.createOrReplaceTempView("stock")

# Run SQL query to filter data
filtered_data = spark.sql("SELECT stock_exchange, company_symbol FROM stock WHERE open_price_of_the_day > 68 AND high_of_the_day < 70")
filtered_data.show()

# Save DataFrame to output folder
output_path = "output1"
df2.write.format('orc').save(output_path)

# Stop SparkSession
spark.stop()


+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|stock_exchange|company_symbol|      date|open_price_of_the_day|high_of_the_day|low_of_the_day|close_of_the_day|   volume|adjusted_close_price|
+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|          NYSE|           CLI|2009-12-31|                35.39|           35.7|          34.5|           34.57| 890100.0|               34.12|
|          NYSE|           CLI|2009-12-30|                35.22|          35.46|         34.96|            35.4| 516900.0|               34.94|
|          NYSE|           CLI|2009-12-29|                35.69|          35.95|         35.21|           35.34| 556500.0|               34.88|
|          NYSE|           CLI|2009-12-28|                35.67|          36.23|         35.49|           35.69| 565000.0|              

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import time
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType
from pyspark.sql.functions import col, struct, when
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark = SparkSession.builder.appName("Rama_StockInfo").getOrCreate()
read_text_data = spark.read.text("/content/NYSE_daily_File")
rowrdd = read_text_data.rdd.map(lambda x : x['value'])
split_rdd = rowrdd.map(lambda x : x.split('\t'))
#employee_details = split_rdd.map(lambda x : (x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7],x[8]))
new_rdd=split_rdd.map(lambda x:(x[0],x[1],x[2],convert_toint(x[3]),convert_toint(x[4]),convert_toint(x[5]),convert_toint(x[6]),convert_toint(x[7]),convert_toint(x[8])))
schema = StructType([
    StructField("stock_exchange",StringType(),True),
    StructField("company_symbol",StringType(),True),
    StructField("date",StringType(),True),
    StructField("open_price_of_the_day",FloatType(),True),
    StructField("high_of_the_day",FloatType(),True),
    StructField("low_of_the_day",FloatType(),True),
    StructField("close_of_the_day",FloatType(),True),
    StructField("volume",FloatType(),True),
    StructField("adjusted_close_price",FloatType(),True)

])
def convert_toint(num1):
    return float(num1)
Stock_df = spark.createDataFrame(new_rdd,schema)
cleaned_Stock_details = Stock_df.dropna()
df2=cleaned_Stock_details.filter("open_price_of_the_day >0 and high_of_the_day >0")
df2.show()

'''
Run Athena query on this table “NYSE_Partition” to find out  which stock’s open price of the day is  greater than 68 and high close price is less than 70.
'''
df2.createOrReplaceTempView("Stock")
res1=spark.sql("select stock_exchange,company_symbol from Stock where open_price_of_the_day>68 and high_of_the_day < 70 ")
output_path = "output1"
Stock_df.write.format('orc').save("output1")

res1.show()
spark.stop()

+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|stock_exchange|company_symbol|      date|open_price_of_the_day|high_of_the_day|low_of_the_day|close_of_the_day|   volume|adjusted_close_price|
+--------------+--------------+----------+---------------------+---------------+--------------+----------------+---------+--------------------+
|          NYSE|           CLI|2009-12-31|                35.39|           35.7|          34.5|           34.57| 890100.0|               34.12|
|          NYSE|           CLI|2009-12-30|                35.22|          35.46|         34.96|            35.4| 516900.0|               34.94|
|          NYSE|           CLI|2009-12-29|                35.69|          35.95|         35.21|           35.34| 556500.0|               34.88|
|          NYSE|           CLI|2009-12-28|                35.67|          36.23|         35.49|           35.69| 565000.0|              

In [ ]:
# Initialize SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg

spark = SparkSession.builder.appName("RamaStock_Price_Variance").getOrCreate()

# Load the data from the output1 folder
data_path = "/content/output1"
df = spark.read.format("orc").load(data_path)

# Calculate the difference between close price and open price
price_diff_df = df.withColumn("price_diff", col("close_of_the_day") - col("open_price_of_the_day"))

# Group by company symbol and calculate the average price difference
average_price_variance = price_diff_df.groupBy("company_symbol").agg(avg("price_diff").alias("average_price_variance"))

#output_path = #"output2"
output_path = "output2"
average_price_variance.write.mode("overwrite").format("orc").save(output_path)

# Display the calculated average price variance
average_price_variance.show()
# Stop the SparkSession
spark.stop()


+--------------+----------------------+
|company_symbol|average_price_variance|
+--------------+----------------------+
|           CSR|  -0.02873014552252...|
|           CCK|   0.02670630197676401|
|           CRS|  0.017182573439582946|
|           CFX|  -0.00178571542104...|
|           CRL|   0.06404772258940197|
|           CJT|   0.05499997632256869|
|            CI|   0.07199206948280334|
|           CWT|  3.176189604259672...|
|           CGX|   0.04686511130560012|
|           CCS|   0.03765879737006293|
|           CHP|  -0.03051587324293...|
|           CUB|  0.020079332684713697|
|           CMN|  6.349086761474609E-4|
|           CNW|   0.03379453877686512|
|           CBI|  -0.02031748824649387|
|           CLH|  0.040078823960672215|
|           CIF|  0.003095240820021...|
|           CPC|  -0.04563911337601511|
|           CRY|  -0.01932539826347...|
|           CLX|  -0.02620544735150846|
+--------------+----------------------+
only showing top 20 rows

